In [13]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import MetaTrader5 as mt5
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sesto.constants import CURRENCY_PAIRS
from sesto.indicators import SMA, BB, RSI
import sesto.metatrader.data as mtd

In [14]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 500
SMA_PERIOD = 14
RSI_OVERSOLD = 20
RSI_OVERBOUGHT = 80

In [15]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Fetched data for pair: USDJPY
Fetched data for pair: EURUSD
Fetched data for pair: GBPUSD
Fetched data for pair: EURGBP
Fetched data for pair: CADCHF
Fetched data for pair: EURJPY
Fetched data for pair: AUDUSD
Fetched data for pair: USDCNH
Fetched data for pair: EURCHF
Fetched data for pair: NZDUSD


In [16]:
for symbol, df in mtd.data.items():
    # Step 1: Calculate the price differences
    df['price_diff'] = df['close'].diff()

    # Step 2: Calculate gains and losses
    df['gain'] = df['price_diff'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['price_diff'].apply(lambda x: -x if x < 0 else 0)
    
    # Step 3: Calculate the average gain and average loss using the smoothing period
    df['avg_gain'] = df['gain'].rolling(window=SMA_PERIOD, min_periods=SMA_PERIOD).mean()
    df['avg_loss'] = df['loss'].rolling(window=SMA_PERIOD, min_periods=SMA_PERIOD).mean()
    
    # Step 4: Calculate the Relative Strength (RS)
    df['rs'] = df['avg_gain'] / df['avg_loss']
    
    # Step 5: Calculate the RSI
    df[f'rsi-{SMA_PERIOD}'] = 100 - (100 / (1 + df['rs']))
    
    # Clean up intermediate columns
    df.drop(columns=['price_diff', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'], inplace=True)
    df.dropna(subset=[f'rsi-{SMA_PERIOD}'], inplace=True)

In [17]:
for symbol, df in mtd.data.items():
        fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True,    # Share the X-axis (date axis) between the plots
        row_heights=[0.7, 0.3],  # Set heights (70% for price, 30% for RSI)
        vertical_spacing=0.15    # Space between the plots
        )

        fig.add_trace(
            go.Scatter(x=df['time'], y=df['close'], mode='lines', name='Price'),
            row=1, col=1
        )

        # Add the RSI line chart to the second subplot
        fig.add_trace(
            go.Scatter(x=df['time'], y=df['rsi-14'], mode='lines', name='RSI', line_color='orange'),
            row=2, col=1
        )

        # Add horizontal lines for RSI thresholds (30 and 70)
        fig.add_hline(y=RSI_OVERBOUGHT, line_dash="dash", line_color="red", row=2, col=1)
        fig.add_hline(y=RSI_OVERSOLD, line_dash="dash", line_color="green", row=2, col=1)

        # Update layout for dark mode and better readability
        fig.update_layout(
            title="Price Line Chart with RSI",
            height=600,  # Adjust the figure height
            template="plotly_dark",  # Apply dark mode template
            xaxis_title="Date",
            yaxis_title="Price",
            yaxis2_title="RSI",
        )

        fig.show()